# MVD 3. cvičení

## 1. část - Stažení a načtení předtrénovaných GloVe slovních reprezentací


### Stažení dat

Pro toto cvičení můžete používat předtrénované Word2Vec nebo GloVe vektory. Následující text se vztahuje ke GloVe vektorům, které byly vybrány z důvodu menší velikosti. 

Základní verzi vektorů lze stáhnout [zde (GloVe link)](https://huggingface.co/stanfordnlp/glove/resolve/main/glove.6B.zip).

Wikipedia 2014 + Gigaword 5 (6B tokens, 400K vocab, uncased, 300d vectors, 822 MB download)

Po rozbalení staženého archivu budete mít několik verzí o různé dimenzi vektorů - 50d, 100d, 200d, 300d. Je doporučeno začít pracovat s nejmenšími vektory a na větších spouštět až závěrečné řešení.

### Načtení dat

Data jsou uložena v textovém souboru, kde je na každém řádku slovo a jeho příslušný vektor.

Načtení je vhodné provést do dvou proměnných -> words, vectors. Words bude list o délce *n* a vectors bude matice o velikosti *(n, d)*. 

Zároveň vytvořte slovník word2idx, pomocí kterého lze získat index libovolného slova (pomocí *word2idx['queen']*).

In [1]:
import numpy as np
VecFile = "glove.6B.300d.txt"
wordVec = list()
vecVec = list()
wordDict = dict()
read = 0

with open(VecFile,"r", encoding = "utf-8") as file:
    if(read>0):
        for i in range(read):
            line  = file.readline()
            line = line.strip().split(" ")
            wordVec.append(line[0])
            wordDict[line[0]] = i
            vecVec.append(np.array(line[1:]).astype(float))
    else:
        i = 0
        for iline in file:
            line = iline.strip().split(" ")
            wordVec.append(line[0])
            wordDict[line[0]] = i
            vecVec.append(np.array(line[1:]).astype(float))
            i += 1
print('king', wordDict["king"])
print('queen', wordDict["queen"])
print('man', wordDict["man"])
print('woman', wordDict["woman"])





king 691
queen 2060
man 300
woman 787


## 2. část - Kosinová podobnost

Vytvořte funkci cossim, která bude vracet kosinovu podobnost dvou vstupních vektorů.

<br>
<center>
$
similarity(a,b) = cos(\theta) = \frac{a \cdot b}{\lVert a \lVert \lVert b \lVert}
$
</center>

In [2]:
def dist(vecl,vecr):
    vv = np.dot(vecl,vecr)
    a = np.sqrt(np.sum(np.square(vecl)))
    b = np.sqrt(np.sum(np.square(vecr)))
    return (vv/(a*b))
print(dist(np.array([1.0,0.0]),np.array([1.1,0.0])))
print(dist(np.array([1.0,0.2]),np.array([0.1,1.0])))
print(dist(np.array([1.0,0.0]),np.array([-0.1,0.0])))

1.0
0.2927142720941118
-1.0


## 3. část - Slovní analogie

Nejznámější slovní analogií je z Word2Vec $f("king") - f("man") = f("queen") - f("woman")$

1. Vytvořte skript pro hledání analogií $f("king") - f("man") = f("??") - f("woman")$ a vyzkoušejte i nějaké další.
2. Vypište 5 nejpodobnějších slov

In [3]:
vec1 = vecVec[wordDict["king"]] - vecVec[wordDict["man"]]
vec2 = vecVec[wordDict["queen"]] - vecVec[wordDict["woman"]]
print(dist(wordDict["king"],wordDict["queen"]))
print(dist(vec1,vec2))
woman = vecVec[wordDict["woman"]]
def find(words:list,biases:list,n = 5):
    topn  = list()
    for i in range(n):
        topn.append((0,0.0))
    unknown = np.zeros(np.shape(vecVec[wordDict[words[0]]]))
    for i,word in enumerate(words):
        if(biases[i]):
            unknown += vecVec[wordDict[word]]
        else:
            unknown -= vecVec[wordDict[word]]
    for key in wordDict:
        d = dist(vecVec[wordDict[key]] ,unknown)
        for i,tup in enumerate(topn):
            if d > tup[1]:
                topn[i] = (wordDict[key],d)
                break
    return topn,unknown
def findPrint(words:list,biases:list,n = 5):
    top1,unknown1 = find(words,biases,n)
    top1.sort(key=lambda tup: tup[1],reverse=True)
    for tup in top1:
        print(wordVec[tup[0]],': ',tup[1])
    return top1,unknown1

1.0
0.6814114764713689


In [4]:
top1,unknown1 = findPrint(["king","woman","man"],[True,True,False],5)

king :  0.8065858062911337
queen :  0.6896163185568701
monarch :  0.5575490901900343
kingdom :  0.5025345084859122
elizabeth :  0.49080310737628424


In [5]:
dist(vecVec[wordDict["king"]],vecVec[wordDict["king"]])

1.0000000000000002

In [6]:
top2,unknown2 = findPrint(["food","sweet"],[True,True],5)

sweet :  0.8021728772131591
foods :  0.6159601532162767
bread :  0.5808867260346312
vegetables :  0.5770362563532005
flavor :  0.5308176510088671


In [7]:
top3,unknown3 = findPrint(["animal","carnivore","herbivore"],[True,True,False],5)

animal :  0.7207073919374275
pet :  0.47079915025802116
wildlife :  0.4413794339654036
carnivore :  0.4357023938353141
elephant :  0.40028740238371774


In [8]:
top4,unknown4 = findPrint(["cat","orange"],[True,True],5)

orange :  0.7729371568438712
cat :  0.7285679973667799
pink :  0.5205206379495432
colored :  0.47718782306253027
cats :  0.4747706727344657


In [9]:
top5,unknown5 = findPrint(["dog","violent"],[True,True],5)

violent :  0.7732826962986239
dog :  0.7675195061164735
dogs :  0.6408122637941089
vicious :  0.5091190971295944
cat :  0.4895154895407166


### Bonus - Vytvořte vizualizaci slovních analogií

Pro získání bonusového bodu je potřeba vytvořit vizualizaci slovních analogií (redukce dimenze + vizualizace).

In [11]:
from sklearn.manifold import TSNE
import numpy as np
import plotly.express as px
x = list()
col = list()
text = list()
def addFound(found,name):
    for tup in found:
        x.append(vecVec[tup[0]])
        col.append(name)
        text.append(wordVec[tup[0]])

addFound(top1,"king-man+woman")
addFound(top2,"food+sweet")
addFound(top3,"animal+carnivour-herbivore")
addFound(top4,"cat+orange")
addFound(top5,"dog+violent")



arr = np.asarray(x)
arr = arr.squeeze()

X_embedded = TSNE(n_components=2,  init='random', perplexity=2).fit_transform(arr)
fig = px.scatter(X_embedded, x=X_embedded[:,0], y=X_embedded[:,1],color = col,text=text)
fig.update_traces(textposition='top center')
fig.show()